In [1]:
import torch
from torch import optim
from torch import nn
from pytorch_pretrained_bert import TransfoXLModel, TransfoXLCorpus, TransfoXLTokenizer
import time, math, sys
sys.path.append('../')
import numpy as np

import models
from utils.data_loader import JSONFileDataLoader
from utils.transf_data_loader import JSONFileDataLoaderTransf
from utils.framework import FewShotREFramework, FewShotREModel
from models.sentence_encoding.basic_sentence_encoder import CNNSentenceEncoder
from models.proto_model import Proto

import warnings
warnings.filterwarnings('ignore', category=UserWarning)

In [2]:
class TransformerSentenceEncoder(nn.Module):
    def __init__(self):
        nn.Module.__init__(self)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        #self.device = "cpu"
        self.model = TransfoXLModel.from_pretrained('transfo-xl-wt103')
        #self.model = self.model.to(self.device)
        #self.model.word_emb.requires_grad = False

    def forward(self, inputs):
        word = inputs.to(self.device)
        print(word.shape)
        l, m = self.model(word, None)
        print(l.shape, len(m))
        return l[-1]

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# tokenizer = TransfoXLTokenizer.from_pretrained('transfo-xl-wt103')
# corpus = TransfoXLCorpus.from_pretrained('transfo-xl-wt103')

# va_iter = corpus.get_iterator('valid', 16, 128, device=device, ext_len=0)
# te_iter = corpus.get_iterator('test', 8, 40, device=device, ext_len=0)

cuda


In [4]:
model_name = 'proto'
N = 5
K = 5
max_length = 40
train_data_loader = JSONFileDataLoaderTransf('../../FewRel/data/train.json',
                                             max_length=max_length, cuda=False)
val_data_loader = JSONFileDataLoaderTransf('../../FewRel/data/val.json',
                                           max_length=max_length, cuda=False)

Loading data file...
Finish loading
Elimiating case sensitive problem...
Finish eliminating
Finish building
Pre-processing data...
Finish pre-processing
Loading data file...
Finish loading
Elimiating case sensitive problem...
Finish eliminating
Finish building
Pre-processing data...
Finish pre-processing


In [5]:
framework = FewShotREFramework(train_data_loader, val_data_loader, None)
encoder = TransformerSentenceEncoder()
model = Proto(encoder)
model = model.cuda()

In [6]:
support, query, label = train_data_loader.next_batch(1, 1, 5, 5)
support_list = []
query_list = []
Iters = support.shape[0] // model.good_batch
if support.shape[0] % model.good_batch == 0:
    Iters += 1
print(Iters, support.shape, query)
for i in range(N):
    batch = support[i*model.good_batch:(i+1)*model.good_batch]
    support_list.append(model.sentence_encoder(batch).cpu())
    print(support_list[-1])
    #break
    batch = query[i*model.good_batch:(i+1)*model.good_batch]
    query_list.append(model.sentence_encoder(batch).cpu())
    print(i)
support = torch.cat(support_list, dim=0)
query = torch.cat(query_list, dim=0)
print(support.shape, query.shape)

0 torch.Size([5, 40]) tensor([[267739,      6,      7,      8,      9, 267735,     14, 267736,     29,
           2750,     49,    300,     27,      1,   2516,     24,    268,   5664,
             83,     24,      5,      1,     24,      3, 267737, 267738, 267738,
         267738, 267738, 267738, 267738, 267738, 267738, 267738, 267738, 267738,
         267738, 267738, 267738, 267738],
        [267739,      7, 267735,     15,     16, 267736,     24,     90,     25,
             40,    729,      4,      1,     24,    559,      7,     24,     14,
             24,    659,     72,     24,     24,     15,      1,    680,    149,
              4,      1,     95,      3, 267737, 267738, 267738, 267738, 267738,
         267738, 267738, 267738, 267738],
        [267739,      7,      8,      9, 267735,     13, 267736,      1,    949,
          12649,     28,   1156,     17,      1,   1140,   9311,   5885,     96,
             59,    879,     24,     24,     15,      1,  10941,     58,    246,
   

RuntimeError: CUDA out of memory. Tried to allocate 96.12 MiB (GPU 0; 11.75 GiB total capacity; 10.26 GiB already allocated; 59.44 MiB free; 99.71 MiB cached)

In [13]:
%%time
support, query, label = train_data_loader.next_batch(1, 7, 1, 5)
encoder(support)

torch.Size([7, 40])
torch.Size([7, 40, 1024]) 18
CPU times: user 141 ms, sys: 180 ms, total: 321 ms
Wall time: 343 ms


In [59]:
support.keys()
query.keys()

dict_keys(['word', 'pos1', 'pos2', 'mask'])

In [61]:
label.shape

torch.Size([20, 25])

In [63]:
support['word'].shape

torch.Size([500, 40])